In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('/content/drive/MyDrive/Colab Notebooks/intents.json').read()
intents = json.loads(intents_file)

In [ ]:
import nltk
nltk.download('punkt')
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'],

In [ ]:
import nltk
nltk.download('wordnet')
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words

    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Training data is created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
20/20 [==============================] - 1s 2ms/step - loss: 3.1124 - accuracy: 0.0909 
Epoch 2/200
20/20 [==============================] - 0s 2ms/step - loss: 2.9635 - accuracy: 0.1111
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 2.8397 - accuracy: 0.1515
Epoch 4/200
20/20 [==============================] - 0s 2ms/step - loss: 2.6504 - accuracy: 0.2222
Epoch 5/200
20/20 [==============================] - 0s 2ms/step - loss: 2.4062 - accuracy: 0.2929
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 2.4037 - accuracy: 0.3030
Epoch 7/200
20/20 [==============================] - 0s 2ms/step - loss: 2.3461 - accuracy: 0.2727
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 2.3071 - accuracy: 0.2828
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 2.1941 - accuracy: 0.3333
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 2.2109 - accuracy: 0.3030
Epoch 11

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/drive/MyDrive/Colab Notebooks/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
#Creating tkinter GUI
# import tkinter
# from tkinter import *
# def send():
#     msg = EntryBox.get("1.0",'end-1c').strip()
#     EntryBox.delete("0.0",END)
#     if msg != '':
#         ChatBox.config(state=NORMAL)
#         ChatBox.insert(END, "You: " + msg + '\n\n')
#         ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
#         ints = predict_class(msg)
#         res = getResponse(ints, intents)
#         ChatBox.insert(END, "Bot: " + res + '\n\n')           
#         ChatBox.config(state=DISABLED)
#         ChatBox.yview(END)
# root = Tk()
# root.title("Chatbot")
# root.geometry("400x500")
# root.resizable(width=FALSE, height=FALSE)
# #Create Chat window
# ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
# ChatBox.config(state=DISABLED)
# #Bind scrollbar to Chat window
# scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
# ChatBox['yscrollcommand'] = scrollbar.set
# #Create Button to send message
# SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
#                     bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
#                     command= send )
# #Create the box to enter message
# EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
# #EntryBox.bind("<Return>", send)
# #Place all components on the screen
# scrollbar.place(x=376,y=6, height=386)
# ChatBox.place(x=6,y=6, height=386, width=370)
# EntryBox.place(x=128, y=401, height=90, width=265)
# SendButton.place(x=6, y=401, height=90)
# root.mainloop()


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('/content/drive/MyDrive/Colab Notebooks/intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=189, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


20/20 [==============================] - 0s 2ms/step - loss: 3.0410 - accuracy: 0.0404
Epoch 2/189
20/20 [==============================] - 0s 2ms/step - loss: 3.0072 - accuracy: 0.0707
Epoch 3/189
20/20 [==============================] - 0s 2ms/step - loss: 2.9809 - accuracy: 0.1010
Epoch 4/189
20/20 [==============================] - 0s 2ms/step - loss: 2.8956 - accuracy: 0.1717
Epoch 5/189
20/20 [==============================] - 0s 2ms/step - loss: 2.8619 - accuracy: 0.1818
Epoch 6/189
20/20 [==============================] - 0s 2ms/step - loss: 2.8050 - accuracy: 0.1717
Epoch 7/189
20/20 [==============================] - 0s 2ms/step - loss: 2.7454 - accuracy: 0.2222
Epoch 8/189
20/20 [==============================] - 0s 2ms/step - loss: 2.6121 - accuracy: 0.2626
Epoch 9/189
20/20 [==============================] - 0s 2ms/step - loss: 2.4885 - accuracy: 0.3131
Epoch 10/189
20/20 [==============================] - 0s 2ms/step - loss: 2.4509 - accuracy: 0.2525
Epoch 11/189
20/20 [=

In [ ]:
!pip install ibm_watson

     |████████████████████████████████| 412 kB 29.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 49 kB 4.9 MB/s 
     |████████████████████████████████| 68 kB 5.6 MB/s 
     |████████████████████████████████| 62 kB 828 kB/s 
     |████████████████████████████████| 138 kB 56.9 MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.3.0-py3-none-any.whl size=408872 sha256=48c7b5f9421989e3af17e3c2e390a523d758e86a80bd6fb859c856717dfb9d86
  Stored in directory: /root/.cache/pip/wheels/21/d9/82/4ce5b94730bc4f1f7b4c6384f72964b9b8f79fcc125bb8085c
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.13.2-py3-none-any.whl size=83241 sha256=0effbd5725b12e8b430687a7dd85d3c33cc84e7f2132f102d75493b68effcc92
  Stored in directory: /root/.cache/pip/wheels/f0/0d/5c/0c26fcc2db712e8d270e52f7c9f6d8abe33ca79ec29438aa14
Successfully built ibm-watson ibm-cloud-sd

In [ ]:
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
msg = list()
text = str()

In [ ]:

def responsed(msg1):
    msg.append(msg1)
    ints = predict_class(msg1)
    res = getResponse(ints, intents)
    return res

In [ ]:

def song_emotion():
    authenticator = IAMAuthenticator("w7uhcCEJ7Pa1CbaKWC0d7Cj4T1r0a3uGbJtzjNx4dMd1")
    tone_analyzer = ToneAnalyzerV3(
        version='2017-09-21',
        authenticator=authenticator
    
    )

    tone_analyzer.set_service_url( "https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/237d6694-ff32-4e2a-a2bf-1de009482942")
    # text = ""
    # for i in msg:
    #     text = text+i
    len1 = len(msg)
    tone_analysis = tone_analyzer.tone(
        {'text': msg[len1-1]+" "+msg[len1-2]+" "+msg[len1-3]+" "+msg[len1-4]+" "+msg[len1-5]},
        content_type='application/json'
    ).get_result()
    dic1 = dict()
    emotion=tone_analysis["document_tone"]["tones"][0]["tone_name"]
    import requests

    url=f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag={emotion}&api_key=d0e2f5a2551c75271acd476e809537b2&format=json&limit=10"
    response = requests.get(url)
    payload = response.json()
    for i in range(10):
        r=payload['tracks']['track'][i]
        dic1[r['name']] = r['url']
    return dic1
    
    
    
   

In [ ]:
print("Chatbot : Hey there, Wassup ?")
# responded function takes text of user and returns chatbot output
for i in range(5):
    m = input("User : ")
    res = responsed(m)
    print("Chatbot : "+res)
ans=song_emotion()













print(ans)

Chatbot : Hey there, Wassup ?
User : Hiii,nice meeting you
Chatbot : Oh yes! I can assure you I am very much alive and conscious!
User : i am fine.. how is your day?
Chatbot : I dont have a name yet but I was thinking maybe Alex. That has a nice ring to it dont you think?
User : dont your owner kept you any name?
Chatbot : I dont have a name yet but I was thinking maybe Alex. That has a nice ring to it dont you think?
User : what is your age?
Chatbot : I'm a robot I dont have an age...
User : What is your movie?
Chatbot : Interstellar was amazing
{'Butter': 'https://www.last.fm/music/BTS/_/Butter', 'You And Your Heart': 'https://www.last.fm/music/Jack+Johnson/_/You+And+Your+Heart', 'Yoshimi Battles The Pink Robots Part 1': 'https://www.last.fm/music/The+Flaming+Lips/_/Yoshimi+Battles+The+Pink+Robots+Part+1', 'Come on! Feel the Illinoise!': 'https://www.last.fm/music/Sufjan+Stevens/_/Come+on%21+Feel+the+Illinoise%21', 'praise you - radio edit': 'https://www.last.fm/music/Fatboy+Slim/_/p

In [ ]:
# song_emotion function would return dictionary consisting of emotion and recommended songs
ans = song_emotion()
lst = list(ans.keys())
print("Song Recommendations : ")
for i in range(10):
    print("Song_name : "+lst[i])
    print("Song_URL : "+ans[lst[i]])

Song Recommendations : 
Song_name : Butter
Song_URL : https://www.last.fm/music/BTS/_/Butter
Song_name : You And Your Heart
Song_URL : https://www.last.fm/music/Jack+Johnson/_/You+And+Your+Heart
Song_name : Yoshimi Battles The Pink Robots Part 1
Song_URL : https://www.last.fm/music/The+Flaming+Lips/_/Yoshimi+Battles+The+Pink+Robots+Part+1
Song_name : Come on! Feel the Illinoise!
Song_URL : https://www.last.fm/music/Sufjan+Stevens/_/Come+on%21+Feel+the+Illinoise%21
Song_name : praise you - radio edit
Song_URL : https://www.last.fm/music/Fatboy+Slim/_/praise+you+-+radio+edit
Song_name : Let's Go
Song_URL : https://www.last.fm/music/Matt+&+Kim/_/Let%27s+Go
Song_name : Old Old Fashioned
Song_URL : https://www.last.fm/music/Frightened+Rabbit/_/Old+Old+Fashioned
Song_name : Lake Shore Drive
Song_URL : https://www.last.fm/music/Aliotta+Haynes+Jeremiah/_/Lake+Shore+Drive
Song_name : My Little Town
Song_URL : https://www.last.fm/music/Simon+&+Garfunkel/_/My+Little+Town
Song_name : Dontcha
Song_